In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime

In [3]:
start_date = "2020-01-01"
end_date = "2023-01-01"
symbol = "BTC-USD"

data = yf.download(symbol, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


# Bollinger Bands & VWAP


In [4]:
data['TP'] = (data['High'] + data['Low'] + data['Close']) / 3
data['TradedValue'] = data['TP'] * data['Volume']
data['CumVol'] = data['Volume'].cumsum()
data['CumTradedValue'] = data['TradedValue'].cumsum()
data['VWAP'] = data['CumTradedValue'] / data['CumVol']

window = 20
data['SMA'] = data['Close'].rolling(window=window).mean()
data['STD'] = data['Close'].rolling(window=window).std()
data['UpperBB'] = data['SMA'] + (2 * data['STD'])
data['LowerBB'] = data['SMA'] - (2 * data['STD'])

In [5]:
data['Long'] = (data['VWAP'] < data['LowerBB']).astype(int)
data['Short'] = (data['VWAP'] > data['UpperBB']).astype(int)
data['Position'] = data['Long'] - data['Short']
data['Returns'] = data['Close'].pct_change() * data['Position'].shift(1)

data['CumReturns'] = (1 + data['Returns']).cumprod()
profitability = (data['CumReturns'].iloc[-1] - 1) * 100

print(f"The profitability of the VWAP and Bollinger Bands trading strategy is {profitability:.2f}%")

The profitability of the VWAP and Bollinger Bands trading strategy is 139.68%
